# <font color='red'> LESSON 3: Datamart Fine-tuning and Model Creation </font>
In this excercise we will perform the following:
  * fine tune our data mart from previous lesson - date feature transformation
    * sweetviz analysis of datamart with focus on distribution and number of 0 values
    * identify features for modification (binning, merging, custom transformations, etc...)
    * prepare final datamart structure
  * model theory (supervised models)
    * <span style="color:red">**Regressors**</span> - for our case linear with numerical label
    * <span style="color:blue">**Classificators**</span> - for our case with binary label

### Household price dataset

Let't work with our dataset and apply knowledge from previous lesson.

Importing raw data and processed dataset from last lesson

In [63]:
import pandas as pd

data = pd.read_parquet('./data/processed_data.parquet')

In [64]:
data = data.set_index('Id')

## <font color='red'> Classifying Data </font>

There are two classes of data, *qualitative* and *quantitative*. 

<font color='blue'> Qualitative data </font> use descriptive terms to differentiate values.
For example, gender is generally classified into "M" or male and "F" or female. Qualitative data can be used for
segmentation or classification. 

<font color='blue'> Quantitative data </font> is characterized by numeric values. 
There are four types of quantitative data.

**Nominal data** is numeric data that represents categories or attributes. The numeric values for gender (1 & 2) would be
nominal data values. One important characteristic of nominal data is that it has no relative importance. For example,
even though male = 1 and female = 2, the relative value of being female is not twice the value or a higher value than that
of being male. For modeling purposes, a nominal variable with only two values would be coded with the values 0 and 1.
This will be discussed in more detail in chapter 4.

**Ordinal data** is numeric data that represents categories that have relative importance. They can be used to rank strength
or severity. For example, a list company assigns the values 1 through 5 to denote financial risk. The value 1,
characterized by no late payments, is considered low risk. The value 5, characterized by a bankruptcy, is considered high
risk. The values 2 through 4 are characterized by various previous delinquencies. A prospect with a risk ranking of 5 is
definitely riskier than a prospect with a ranking of 1. But he or she is not five times as risky. And the difference in their
ranks of 5 –1 = 4 has no meaning.

**Interval data** is numeric data that has relative importance and has no zero point. Also, addition and subtraction are
meaningful operations. For example, many financial institutions use a risk score that has a much finer definition than the
values 1 through 5, as in our previous example. A typical range is from 300 to 800. It is therefore possible to compare
scores by measuring the difference.

**Continuous data** is the most common data used to develop predictive models. It can accommodate all basic arithmetic
operations, including addition, subtraction, multiplication, and division. 
Most business data such as sales, balances, and minutes, is continuous data.

 
![feature types](https://cdn.shopify.com/s/files/1/1334/2321/articles/Picture1.png?v=1497575369)

## <font color='red'>Categorical Features </font>



  - **Categorical feature**: Categorical variables contain a finite number of categories or distinct groups. Categorical data might not have a logical order. *For example, categorical predictors include gender, material type, and payment method.*

### Feature split

In [65]:
from pandas.api.types import is_numeric_dtype

unique_count_ratio = 0.05
empty_ratio = 0.9
min_distinct_values = 2
columns = data.columns
# define label columns
label = 'SalePrice'
date_features = ['YearBuilt', 'GarageYrBlt', 'YearRemodAdd', 'YrSold']
# manually remove features
drop_features = ['PriceNtile', 'Neighbourhood_Rank']
# define empty list of categorical features which we will fill
cat_features = []
# define empty list of numerical features which we will fill
num_features = []

print(columns)

#consider feature categorical if ratio is below given value and number of unique records is >= 2
for var in columns:
    if \
      1.*data[var].nunique()/data[var].count() < unique_count_ratio \
      and data[var].nunique() >= min_distinct_values \
      and data[var].isna().sum() / data[var].count() < empty_ratio:
        cat_features.append(var)
    
#consider feature numerical if ratio is greater than or equal to given value and feature has numerival data type
for var in columns:
    if \
      1.*data[var].nunique()/data[var].count() >= unique_count_ratio \
      and is_numeric_dtype(data[var]) \
      and data[var].isna().sum() / data[var].count() < empty_ratio:
        num_features.append(var)
        
# remove data features from feature lists
cat_features = [f for f in cat_features if f not in date_features + drop_features ]
num_features = [f for f in num_features if f not in date_features + drop_features ]
leftover_features = [c for c in columns if c not in num_features + cat_features + date_features]
display(leftover_features)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'Neighbourhood_Rank', 'PriceNtile']

Based on the logic explained above:
* the variables in dictionary with value *True* are considered categorical
* the inappropriate variables are not in list anymore 

### Date derived variables

In [66]:
date_data = data[date_features]
display(date_data)

,YearBuilt,GarageYrBlt,YearRemodAdd,YrSold
Id,,,,
1,2003,2003.0,2003,2008
2,1976,1976.0,1976,2007
3,2001,2001.0,2002,2008
4,1915,1998.0,1970,2006
5,2000,2000.0,2000,2008
...,...,...,...,...
1456,1999,1999.0,2000,2007
1457,1978,1978.0,1988,2010
1458,1941,1941.0,2006,2010


In [73]:
import numpy as np
pd.options.mode.chained_assignment = None
date_data = data[date_features]

for c in date_data:
    max_val = date_data[c].max()
    print("max for {c} is : {v}".format(c=c, v=max_val))
    date_data[c + "_age"] =  np.where(date_data[c] == 0, 0, max_val - date_data[c]).astype(float)
    

bin_labels_5 = ['Newest', 'New', 'Mid', 'Old', 'Oldest']
date_data['YearBuilt_b'] = pd.qcut(date_data['YearBuilt_age'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5)
date_data['GarageYrBlt_b'] = pd.qcut(date_data['GarageYrBlt_age'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5)
date_data['YearRemodAdd_b'] = pd.qcut(date_data['YearRemodAdd_age'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5)
age_mapping = {0: 'Newly',
           1: 'Recently',
           2: 'Earlier',
           3: 'Older',
           4: 'Longago'
               }
date_data['YrSold_b'] = date_data.YrSold_age.map(age_mapping)
for c in date_features:
    date_data[c + "_b"] =  np.where(date_data[c].isnull(), 'UNKNOWN', date_data[c + "_b"]).astype(str)
display(date_data)

date_data = date_data[[x + '_b' for x in date_features]]

max for YearBuilt is : 2010
max for GarageYrBlt is : 2010.0
max for YearRemodAdd is : 2010
max for YrSold is : 2010


,YearBuilt,GarageYrBlt,YearRemodAdd,YrSold,YearBuilt_age,GarageYrBlt_age,YearRemodAdd_age,YrSold_age,YearBuilt_b,GarageYrBlt_b,YearRemodAdd_b,YrSold_b
Id,,,,,,,,,,,,
1,2003,2003.0,2003,2008,7.0,7.0,7.0,2.0,Newest,New,New,Earlier
2,1976,1976.0,1976,2007,34.0,34.0,34.0,3.0,Mid,Mid,Old,Older
3,2001,2001.0,2002,2008,9.0,9.0,8.0,2.0,New,New,New,Earlier
4,1915,1998.0,1970,2006,95.0,12.0,40.0,4.0,Oldest,New,Old,Longago
5,2000,2000.0,2000,2008,10.0,10.0,10.0,2.0,New,New,New,Earlier
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,1999,1999.0,2000,2007,11.0,11.0,10.0,3.0,New,New,New,Older
1457,1978,1978.0,1988,2010,32.0,32.0,22.0,0.0,Mid,Mid,Mid,Newly
1458,1941,1941.0,2006,2010,69.0,69.0,4.0,0.0,Oldest,Oldest,Newest,Newly


### Another derived variables

In [75]:
has_garage_threshold = 120
date_data['HasGarage'] = np.where(date_data['GarageYrBlt_b'] == 'UNKNOWN', False, True).astype(bool)
display(date_data)
display(date_data['HasGarage'].value_counts())

,YearBuilt_b,GarageYrBlt_b,YearRemodAdd_b,YrSold_b,HasGarage
Id,,,,,
1,Newest,New,New,Earlier,True
2,Mid,Mid,Old,Older,True
3,New,New,New,Earlier,True
4,Oldest,New,Old,Longago,True
5,New,New,New,Earlier,True
...,...,...,...,...,...
1456,New,New,New,Older,True
1457,Mid,Mid,Mid,Newly,True
1458,Oldest,Oldest,Newest,Newly,True


True     1379
False      81
Name: HasGarage, dtype: int64

In [ ]:
###

In [74]:
cat_data = data[cat_features].astype(str)
display(cat_features)
cat_data

['MSSubClass',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 '3SsnPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'SaleType',
 'SaleCondition']

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageCars,GarageQual,GarageCond,PavedDrive,3SsnPorch,PoolArea,MiscVal,MoSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,2,TA,TA,Y,0,0,0,2,WD,Normal
2,20,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,2,TA,TA,Y,0,0,0,5,WD,Normal
3,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,2,TA,TA,Y,0,0,0,9,WD,Normal
4,70,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,3,TA,TA,Y,0,0,0,2,WD,Abnorml
5,60,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,3,TA,TA,Y,0,0,0,12,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,2,TA,TA,Y,0,0,0,8,WD,Normal
1457,20,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,2,TA,TA,Y,0,0,0,2,WD,Normal
1458,70,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,1,TA,TA,Y,0,0,2500,5,WD,Normal


## <font color='red'> Transformation of Categorical Features </font>


Add parsed date data to categorical data 

In [76]:
cat_data = pd.merge(cat_data, date_data, how='inner', left_index=True, right_index=True)
display(cat_data)

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,PoolArea,MiscVal,MoSold,SaleType,SaleCondition,YearBuilt_b,GarageYrBlt_b,YearRemodAdd_b,YrSold_b,HasGarage
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0,0,2,WD,Normal,Newest,New,New,Earlier,True
2,20,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,0,0,5,WD,Normal,Mid,Mid,Old,Older,True
3,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0,0,9,WD,Normal,New,New,New,Earlier,True
4,70,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,0,0,2,WD,Abnorml,Oldest,New,Old,Longago,True
5,60,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,0,0,12,WD,Normal,New,New,New,Earlier,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,0,0,8,WD,Normal,New,New,New,Older,True
1457,20,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,0,0,2,WD,Normal,Mid,Mid,Mid,Newly,True
1458,70,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,0,2500,5,WD,Normal,Oldest,Oldest,Newest,Newly,True


### Find number of occurences for categories

In [77]:
import numpy as np
pd.set_option('display.max_rows', 500)

pivot_data = cat_data.agg(['nunique']).T
pivot_data['over_8'] = np.where(pivot_data['nunique'] > 8, 1, 0).astype(int)

dict = {}
for x in cat_data.columns:
    value_count = cat_data[x].value_counts()
    value_count = value_count/cat_data[x].count()
    dict[x] = value_count.where(lambda x : x < 0.05).count()
    
result = pd.merge(pivot_data, pd.Series(dict, name='under_5_perc'), left_index=True, right_index=True)
result['flag'] = np.where((result['over_8'] > 0) | (result['under_5_perc'] > 0), 1, 0)
display(result)

,nunique,over_8,under_5_perc,flag
MSSubClass,15,1,11,1
MSZoning,5,0,3,1
Street,2,0,1,1
LotShape,4,0,2,1
LandContour,4,0,3,1
Utilities,2,0,1,1
LotConfig,5,0,2,1
LandSlope,3,0,2,1
Neighborhood,25,1,16,1
Condition1,9,1,7,1


### Replace categories of minor occurences

In [79]:
df_size = cat_data.shape[0]
min_percentage_to_merge = 0.05
replaced_cat_data = cat_data.copy()

#iterate through all categorical features
for f in cat_data:
    #group by category value and get count
    cat_distribution = pd.DataFrame(cat_data[[f]].groupby(f).size() / df_size).reset_index().rename(columns={0: 'count'}).sort_values(by='count')
    #get those values with count <= that minimum value
    insufficient_count_data = cat_distribution[cat_distribution['count'] <= min_percentage_to_merge]
    #if there are more of those per category
    if insufficient_count_data.shape[0] > 1:
        display(insufficient_count_data)
        replaced_cat_data[f] = replaced_cat_data[f].replace(insufficient_count_data[f].tolist(), 'OTHER')
    
display(replaced_cat_data)

,MSSubClass,count
6,40,0.002740
2,180,0.006849
7,45,0.008219
11,75,0.010959
13,85,0.013699
3,190,0.020548
14,90,0.035616
12,80,0.039726
10,70,0.041096
1,160,0.043151


,MSZoning,count
0,C (all),0.006849
2,RH,0.010959
1,FV,0.044521


,LotShape,count
2,IR3,0.006849
1,IR2,0.028082


,LandContour,count
2,Low,0.024658
1,HLS,0.034247
0,Bnk,0.043151


,LotConfig,count
3,FR3,0.002740
2,FR2,0.032192


...

,MiscVal,count
20,8300,0.000685
15,560,0.000685
14,54,0.000685
19,800,0.000685
17,620,0.000685
8,350,0.000685
7,2500,0.000685
9,3500,0.000685
5,15500,0.000685
4,1400,0.000685


,MoSold,count
4,2,0.035616
0,1,0.039726
3,12,0.040411
11,9,0.043151


,SaleType,count
2,Con,0.001370
7,Oth,0.002055
1,CWD,0.002740
4,ConLI,0.003425
5,ConLw,0.003425
3,ConLD,0.006164
0,COD,0.029452


,SaleCondition,count
1,AdjLand,0.002740
2,Alloca,0.008219
3,Family,0.013699


,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,PoolArea,MiscVal,MoSold,SaleType,SaleCondition,YearBuilt_b,GarageYrBlt_b,YearRemodAdd_b,YrSold_b,HasGarage
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0,0,OTHER,WD,Normal,Newest,New,New,Earlier,True
2,20,RL,Pave,Reg,Lvl,AllPub,OTHER,Gtl,OTHER,Feedr,...,0,0,5,WD,Normal,Mid,Mid,Old,Older,True
3,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0,0,OTHER,WD,Normal,New,New,New,Earlier,True
4,OTHER,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,OTHER,Norm,...,0,0,OTHER,WD,Abnorml,Oldest,New,Old,Longago,True
5,60,RL,Pave,IR1,Lvl,AllPub,OTHER,Gtl,OTHER,Norm,...,0,0,OTHER,WD,Normal,New,New,New,Earlier,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,0,0,8,WD,Normal,New,New,New,Older,True
1457,20,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,OTHER,Norm,...,0,0,OTHER,WD,Normal,Mid,Mid,Mid,Newly,True
1458,OTHER,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,OTHER,Norm,...,0,OTHER,5,WD,Normal,Oldest,Oldest,Newest,Newly,True


###  One-Hot encoding

  - One-hot encoding is an extremely common data transformation performed after indexing categorical variables. This is because indexing does not always represent our categorical variables in the correct way for downstream models to process. For instance, when we index our “color” column, you will notice that some colors have a higher value (or index number) than others (in our case, blue is 1 and green is 2).
  - This is incorrect because it gives the mathematical appearance that the input to the machine learning algorithm seems to specify that green > blue, which makes no sense in the case of the current categories. To avoid this, we use OneHotEncoder, which will convert each distinct value to a Boolean flag (1 or 0) as a component in a vector. When we encode the color value, then we can see these are no longer ordered, making them easier for downstream models (e.g., a linear model) to process
  - Input column must be of type NumericType
  


Pandas get_dummies() converts categorical variables into dummy variables

In [80]:
oh_data = replaced_cat_data.copy()
oh_data = pd.get_dummies(oh_data, columns=oh_data.columns.tolist(), prefix = oh_data.columns.tolist())

display(oh_data.head())

,MSSubClass_120,MSSubClass_20,MSSubClass_50,MSSubClass_60,MSSubClass_OTHER,MSZoning_OTHER,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,...,YearRemodAdd_b_Newest,YearRemodAdd_b_Old,YearRemodAdd_b_Oldest,YrSold_b_Earlier,YrSold_b_Longago,YrSold_b_Newly,YrSold_b_Older,YrSold_b_Recently,HasGarage_False,HasGarage_True
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1
2,0,1,0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,1,0,0,1
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,0,0,0,1
5,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1


# Putting it all together

Creating a full datamart that contains all attributes - both categorical and numerical as processed.

In [82]:
scaled_data = pd.read_csv('./data/scaled_data.csv')
full_data = pd.merge(oh_data, scaled_data.drop('SalePrice', axis=1), how='inner', left_index=True, right_index=True)
full_data = pd.merge(full_data, data[['SalePrice']], how='inner', left_index=True, right_index=True).rename(columns={'SalePrice': 'label'})
display(full_data)

,MSSubClass_120,MSSubClass_20,MSSubClass_50,MSSubClass_60,MSSubClass_OTHER,MSZoning_OTHER,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,...,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,label
1,0,0,0,1,0,0,1,0,0,1,...,0.532289,0.289638,-0.796041,-0.502349,-0.051541,1.768105,-0.811747,-0.359325,-0.270208,208500
2,0,1,0,0,0,0,1,0,0,1,...,-0.327437,-0.653917,1.191828,0.586571,0.663315,-0.787243,-0.011497,-0.359325,-0.270208,181500
3,0,0,0,1,0,0,1,0,0,1,...,-0.739702,-0.540801,0.939328,0.443182,0.827539,-0.787243,-0.144872,4.092524,-0.270208,223500
4,0,0,0,0,1,0,1,0,0,1,...,0.238172,-0.033157,1.621080,1.442744,1.764579,0.859156,0.788753,-0.359325,-0.270208,140000
5,0,0,0,1,0,0,1,0,0,1,...,-0.639150,-0.996025,0.503190,-0.294540,0.045061,-0.444243,-0.240140,-0.359325,-0.270208,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,1,0,0,0,1,0,0,0,1,...,-0.244481,-0.562873,0.797009,0.297716,-0.051541,-0.787243,-0.049605,-0.359325,-0.270208,185000
1456,0,0,0,1,0,0,1,0,0,1,...,1.236157,2.527134,-0.796041,1.182983,0.141664,2.205430,-0.811747,-0.359325,-0.270208,175000
1457,0,1,0,0,0,0,1,0,0,1,...,0.255769,0.085477,1.848330,1.737833,-1.056203,-0.787243,0.331467,-0.359325,-0.270208,210000
1458,0,0,0,0,1,0,1,0,0,1,...,0.069746,-0.218006,-0.796041,-0.884718,-1.114164,2.351205,-0.811747,1.473789,-0.270208,266500


In [83]:
import sweetviz as sv
report = sv.analyze(source=[full_data, "Datamart v2"],
            target_feat = 'label',
            pairwise_analysis = 'on')

                                             |          | [  0%]   00:00 -> (? left)

In [84]:
report.show_html('report_new.html')

Report report_new.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [85]:
full_data.to_excel('./data/full_data.xlsx')
full_data.to_parquet('./data/full_data.parquet')

### Feature combination to get new features

In [86]:
#Feature combination

overall_qual_vals = data['OverallQual'].unique()
exter_qual_vals = data['ExterQual'].unique()
comb_data = pd.DataFrame()

for x in overall_qual_vals:
    for y in exter_qual_vals:
        comb_data['Overall_qual_' + str(x) + '<-> Exter_qual_' + y] = (data['OverallQual'] == x) & (data['ExterQual'] == y)
        
comb_data = pd.merge(comb_data.astype(int),full_data[['label']], how='inner', left_index=True, right_index=True)

comb_data

,Overall_qual_7<-> Exter_qual_Gd,Overall_qual_7<-> Exter_qual_TA,Overall_qual_7<-> Exter_qual_Ex,Overall_qual_7<-> Exter_qual_Fa,Overall_qual_6<-> Exter_qual_Gd,Overall_qual_6<-> Exter_qual_TA,Overall_qual_6<-> Exter_qual_Ex,Overall_qual_6<-> Exter_qual_Fa,Overall_qual_8<-> Exter_qual_Gd,Overall_qual_8<-> Exter_qual_TA,...,Overall_qual_3<-> Exter_qual_Fa,Overall_qual_1<-> Exter_qual_Gd,Overall_qual_1<-> Exter_qual_TA,Overall_qual_1<-> Exter_qual_Ex,Overall_qual_1<-> Exter_qual_Fa,Overall_qual_2<-> Exter_qual_Gd,Overall_qual_2<-> Exter_qual_TA,Overall_qual_2<-> Exter_qual_Ex,Overall_qual_2<-> Exter_qual_Fa,label
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,208500
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,181500
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,223500
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,140000
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,185000
1456,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,175000
1457,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,210000
1458,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,266500


In [87]:
import sweetviz as sv
report = sv.analyze(source=[comb_data, "Combined features"],
            target_feat = 'label',
            pairwise_analysis = 'on')

                                             |          | [  0%]   00:00 -> (? left)

In [88]:
report.show_html('report_combination.html')

Report report_combination.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
